In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import seaborn as sns
pd.set_option('display.max_columns',None)
from sklearn.metrics import roc_auc_score
import pickle
import dill
TARGET = 'answered_correctly'

In [25]:
with open('../features/all_data/loop_feats6.dill','rb') as f:
    d = dill.load(f)

In [26]:
d['user_list']

defaultdict(list,
            {115: [46, 32, 896999.0, 6, 5692],
             124: [30, 7, 544997.0, 0, 7900],
             2746: [19, 11, 325000.0, 11, 5273],
             5382: [125, 84, 4470000.0, 113, 5000],
             8623: [109, 70, 2819600.0, 96, 3915],
             8701: [17, 10, 342000.0, 9, 3901],
             12741: [265, 152, 10516417.0, 250, 5145],
             13134: [1243, 878, 26853839.0, 1233, 3926],
             24418: [6283, 4337, 134463967.0, 6251, 7900],
             24600: [50, 17, 1113248.0, 19, 7900],
             32421: [30, 14, 791998.0, 0, 7900],
             40828: [92, 58, 2260665.0, 61, 7900],
             44331: [291, 171, 9039215.0, 280, 5542],
             46886: [44, 27, 741000.0, 35, 5059],
             50132: [74, 46, 1510000.0, 61, 4485],
             51285: [22, 12, 460000.0, 12, 5692],
             53842: [30, 7, 543997.0, 0, 7900],
             81002: [17, 9, 280000.0, 9, 3802],
             81429: [30, 14, 492998.0, 0, 7900],
             9121

In [41]:
base = pd.read_feather('../features/all_data/BASE_FIX_valid.feather')
loop = pd.read_feather('../features/all_data/LOOP_FIX_TIME2_valid.feather')
loop2 = pd.read_feather('../features/all_data/ROLLING_MEAN2_valid.feather')

In [42]:
df = pd.concat([base,loop,loop2],axis=1)

In [43]:
del base,loop

In [44]:
df.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,viretual_time_stamp,bundle_id,correct_answer,part,tags,tags1,tags2,tags3,tags4,tags5,tags6,community,part_community,ans_user_avg,ans_user_count,elapsed_time_user_avg,elapsed_time_user_sum,explanation_user_avg,first_bundle,ans_content_avg,elapsed_time_content_avg,explanation_content_avg,lag_time_1,lag_time_2,lag_time_3,lag_incorrect_time,lag_part_time_1,lag_part_time_2,lag_part_time_3,ans_user_part_avg,user_content_count,user_part_count,rolling_mean_10,rolling_mean_3
0,8805,0,45001,7900,0,0,3,0,NaN,0,86008635065,7900,0,1,131 93 81,131.0,93.0,81.0,NaN,NaN,NaN,2,12,NaN,0.0,NaN,NaN,NaN,7900.0,0.900000,4400.000000,0.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,8806,21730,45001,7876,0,1,0,0,24000.0,0,86008656795,7876,3,1,10 94 92,10.0,94.0,92.0,NaN,NaN,NaN,2,12,0.000000,1.0,0.000000,0.0,0.0,7900.0,0.307692,23000.000000,0.230769,21730.0,NaN,NaN,21730.0,21730.0,NaN,NaN,0.0,0.0,1.0,NaN,NaN
2,8807,42763,45001,175,0,2,2,1,19000.0,0,86008677828,175,2,1,9 10 92,9.0,10.0,92.0,NaN,NaN,NaN,2,12,0.000000,2.0,12000.000000,24000.0,0.0,7900.0,0.461538,24092.308594,0.307692,21033.0,42763.0,NaN,21033.0,21033.0,42763.0,NaN,0.0,0.0,2.0,NaN,NaN
3,8808,61018,45001,1278,0,3,1,0,18000.0,0,86008696083,1278,3,2,143 140 81 29,143.0,140.0,81.0,29.0,NaN,NaN,2,22,0.333333,3.0,14333.333008,43000.0,0.0,7900.0,0.777778,22000.000000,0.000000,18255.0,39288.0,61018.0,39288.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.333333
4,8809,102918,45001,2065,0,4,0,0,15000.0,0,86008737983,2063,2,3,136 162 92 29,136.0,162.0,92.0,29.0,NaN,NaN,2,32,0.250000,4.0,15250.000000,61000.0,0.0,7900.0,0.727273,5000.000000,0.090909,41900.0,60155.0,81188.0,41900.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.333333


In [45]:
df[df['user_id'] == 45001][['user_id','part','answered_correctly','first_bundle','ans_user_count','ans_content_avg','prior_question_elapsed_time',\
    'prior_question_elapsed_time','elapsed_time_user_sum','elapsed_time_user_avg','bundle_id']][:100]

,user_id,part,answered_correctly,first_bundle,ans_user_count,ans_content_avg,prior_question_elapsed_time,prior_question_elapsed_time,elapsed_time_user_sum,elapsed_time_user_avg,bundle_id
0,45001,1,0,7900.0,0.0,0.900000,NaN,NaN,NaN,NaN,7900
1,45001,1,0,7900.0,1.0,0.307692,24000.0,24000.0,0.0,0.000000,7876
2,45001,1,1,7900.0,2.0,0.461538,19000.0,19000.0,24000.0,12000.000000,175
3,45001,2,0,7900.0,3.0,0.777778,18000.0,18000.0,43000.0,14333.333008,1278
4,45001,3,0,7900.0,4.0,0.727273,15000.0,15000.0,61000.0,15250.000000,2063
5,45001,3,1,7900.0,4.0,0.000000,15000.0,15000.0,61000.0,15250.000000,2063
6,45001,3,0,7900.0,4.0,0.545455,15000.0,15000.0,61000.0,15250.000000,2063
7,45001,4,0,7900.0,7.0,0.444444,13000.0,13000.0,76000.0,10857.142578,3363
8,45001,4,0,7900.0,7.0,0.222222,13000.0,13000.0,76000.0,10857.142578,3363
9,45001,4,0,7900.0,7.0,0.111111,13000.0,13000.0,76000.0,10857.142578,3363


In [32]:
df['user_id'][:10000].unique()[:20]

array([  45001,  176102,  238966,  260489,  357865,  364452,  459017,
        592864,  616513,  653826,  702591, 1024273, 1108398, 1186307,
       1298591, 1321608, 1547989, 1647850, 1857371, 1960671])

In [37]:
df[df['user_id'] == 45001]['task_container_id'].value_counts()

15    4
14    4
7     3
6     3
5     3
4     3
13    1
12    1
11    1
10    1
9     1
8     1
3     1
2     1
1     1
0     1
Name: task_container_id, dtype: int64

In [21]:
df[['user_id','bundle_id','content_id','timestamp','lag_time_1','lag_time_2','lag_time_3','elapsed_time_user_sum','ans_user_count','user_content_count']].iloc[260:280]

,user_id,bundle_id,content_id,timestamp,lag_time_1,lag_time_2,lag_time_3,elapsed_time_user_sum,ans_user_count,user_content_count
260,8623,10688,10688,172041830,47966.0,110165.0,155028.0,846000.0,40.0,0.0
261,8623,675,675,172087052,45222.0,93188.0,155387.0,862000.0,41.0,1.0
262,8623,651,651,172110923,23871.0,69093.0,117059.0,880000.0,42.0,1.0
263,8623,7744,7747,320281387,148170464.0,148194336.0,148239552.0,897000.0,43.0,0.0
264,8623,7744,7748,320281387,148170464.0,148194336.0,148239552.0,914000.0,43.0,0.0
265,8623,7744,7745,320281387,148170464.0,148194336.0,148239552.0,931000.0,43.0,0.0
266,8623,7744,7744,320281387,148170464.0,148194336.0,148239552.0,948000.0,43.0,0.0
267,8623,7744,7746,320281387,148170464.0,148194336.0,148239552.0,965000.0,43.0,0.0
268,8623,7190,7193,331424277,11142890.0,159313360.0,159337232.0,982000.0,48.0,0.0
269,8623,7190,7190,331424277,11142890.0,159313360.0,159337232.0,1059000.0,48.0,0.0


In [15]:
a = [1,2,3]
for i,j in enumerate(a):
    i,j

In [16]:
i

2